In [1]:
print("ok")

ok


In [3]:
import os
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import numpy as np
import pandas as pd
import asyncio
import json

from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from ragas import SingleTurnSample, EvaluationDataset, evaluate
from ragas.metrics import (
    Faithfulness,
    ResponseRelevancy,
    LLMContextPrecisionWithReference,
    LLMContextRecall,
    ContextEntityRecall,
    NoiseSensitivity
)

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

print("✅ All imports successful")

✅ All imports successful


In [5]:
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
)

ragas_llm = LangchainLLMWrapper(llm)
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

print("✅ LLM initialized: gpt-4o")
print("✅ Embeddings initialized: text-embedding-ada-002")
print("✅ RAGAS wrappers ready")

✅ LLM initialized: gpt-4o
✅ Embeddings initialized: text-embedding-ada-002
✅ RAGAS wrappers ready


In [19]:
def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio
            nest_asyncio.apply()
            return loop.run_until_complete(coro)
        else:
            return asyncio.run(coro)
    except RuntimeError:
        return asyncio.run(coro)


In [7]:
test_response = "The first Super Bowl was held on January 15, 1967 in Los Angeles. It was a sunny day with clear skies."
test_context = [
    "The First AFL-NFL World Championship Game was played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles, California."
]
print("📝 Response to evaluate:")
print(f"   '{test_response}'")
print("\n📚 Retrieved context:")
print(f"   '{test_context[0]}'")

📝 Response to evaluate:
   'The first Super Bowl was held on January 15, 1967 in Los Angeles. It was a sunny day with clear skies.'

📚 Retrieved context:
   'The First AFL-NFL World Championship Game was played on January 15, 1967, at the Los Angeles Memorial Coliseum in Los Angeles, California.'


In [8]:
claim_extraction_prompt = ChatPromptTemplate.from_template("""
Given the following response, extract ALL factual claims as a numbered list.
Each claim should be a single, verifiable statement.

Response: {response}

Extract each factual claim:
""")

claim_chain = claim_extraction_prompt | llm | StrOutputParser()
extracted_claim_raw = claim_chain.invoke({"response":test_context})

print("🔍 STEP 1: Extracted Claims from Response")
print("=" * 50)
print(extracted_claim_raw)

🔍 STEP 1: Extracted Claims from Response
1. The First AFL-NFL World Championship Game was played on January 15, 1967.  
2. The game took place at the Los Angeles Memorial Coliseum.  
3. The game was held in Los Angeles, California.


In [9]:
claims = [
    "The first Super Bowl was held on January 15, 1967",
    "The first Super Bowl was held in Los Angeles",
    "It was a sunny day",
    "There were clear skies"
]
print("📋 Claims to verify:")
for i, claim in enumerate(claims, 1):
    print(f"   {i}. {claim}")

📋 Claims to verify:
   1. The first Super Bowl was held on January 15, 1967
   2. The first Super Bowl was held in Los Angeles
   3. It was a sunny day
   4. There were clear skies


In [12]:
verification_prompt = ChatPromptTemplate.from_template("""
Given the following context and claim, determine if the claim is SUPPORTED by the context.

Context: {context}

Claim: {claim}

Answer with:
- "SUPPORTED" if the claim can be verified from the context
- "NOT SUPPORTED" if the claim cannot be verified or contradicts the context

Also provide a brief explanation.

Verdict:
""")

verify_claims = verification_prompt | llm | StrOutputParser()

print("🔍 STEP 2: Verifying Each Claim Against Context")
print("=" * 60)

verification_results = []
for claim in claims:
    result = verify_claims.invoke({
        "context":test_context[0],
        "claim":claim
    })
    is_supported = "SUPPORTED" in result.upper() and "NOT SUPPORTED" not in result.upper()
    verification_results.append({
        "claim":claim,
        "supported": is_supported,
        "explanation":result,
    })
    status = "✅" if is_supported else "❌"
    print(f"\n{status} Claims: {claim}")
    print(f"    Result: {result[:100]}..." if len(result)>100 else f"    Result: {result[:100]}")

🔍 STEP 2: Verifying Each Claim Against Context

✅ Claims: The first Super Bowl was held on January 15, 1967
    Result: **SUPPORTED**

**Explanation:** The context states that the First AFL-NFL World Championship Game wa...

✅ Claims: The first Super Bowl was held in Los Angeles
    Result: SUPPORTED

Explanation: The context states that the First AFL-NFL World Championship Game (the event...

❌ Claims: It was a sunny day
    Result: **NOT SUPPORTED**

**Explanation:** The context provides information about the date, location, and e...

❌ Claims: There were clear skies
    Result: NOT SUPPORTED

Explanation: The context does not provide any information about the weather condition...


In [13]:
print("\n📊 Claim Verification Summary")
print("=" * 80)

df_verification = pd.DataFrame([
    {
        "Clain": r['claim'],
        "Supported?":"✅ Yes" if r["supported"] else "❌ No",
        "Reason":"Found in context" if r['supported'] else "HALLUCINATION - Not in context!"

    }
    for r in verification_results
])
df_verification.head()


📊 Claim Verification Summary


,Clain,Supported?,Reason
0,"The first Super Bowl was held on January 15, 1967",✅ Yes,Found in context
1,The first Super Bowl was held in Los Angeles,✅ Yes,Found in context
2,It was a sunny day,❌ No,HALLUCINATION - Not in context!
3,There were clear skies,❌ No,HALLUCINATION - Not in context!


In [17]:
supported_count = sum(1 for r in verification_results if r['supported'])
total_claims = len(verification_results)

manual_faithfulness = supported_count/total_claims

print("🔢 STEP 3: Calculate Faithfulness Score")
print("=" * 50)
print(f"\n   Supported claims: {supported_count}")
print(f"   Total claims: {total_claims}")
print(f"\n   Formula: Faithfulness = {supported_count} / {total_claims}")
print(f"\n   📊 Manual Faithfulness Score: {manual_faithfulness:.2f}")

🔢 STEP 3: Calculate Faithfulness Score

   Supported claims: 2
   Total claims: 4

   Formula: Faithfulness = 2 / 4

   📊 Manual Faithfulness Score: 0.50


In [ ]:
faithfulness_sample = SingleTurnSample(
    user_input="When was the first Super Bowl?",
    response=test_response,
    retrieved_contexts=test_context
)
faithfulness_metric = Faithfulness(llm=ragas_llm)
ragas_faithfulness = run_async(faithfulness_metric.single_turn_ascore(faithfulness_sample))

print("🔬 RAGAS Faithfulness Result")
print("=" * 50)
print(f"\n   Manual calculation:  {manual_faithfulness:.2f}")
print(f"   RAGAS metric score:  {ragas_faithfulness:.2f}")
print(f"\n   Difference: {abs(manual_faithfulness - ragas_faithfulness):.2f}")

🔬 RAGAS Faithfulness Result

   Manual calculation:  0.50
   RAGAS metric score:  0.50

   Difference: 0.00


In [25]:
faithfulness_examples = [
    {
        "name": "Perfect Faithfulness (No hallucinations)",
        "response": "The first Super Bowl was played on January 15, 1967 at the Los Angeles Memorial Coliseum.",
        "context": ["The First AFL-NFL World Championship Game was played on January 15, 1967, at the Los Angeles Memorial Coliseum."]
    },
    {
        "name": "Partial Faithfulness (Some hallucinations)",
        "response": "The first Super Bowl was on January 15, 1967. The Green Bay Packers won 35-10 with Bart Starr as MVP.",
        "context": ["The First AFL-NFL World Championship Game was played on January 15, 1967."]
    },
    {
        "name": "Zero Faithfulness (Complete hallucination)",
        "response": "The first Super Bowl was held in Miami in 1970 and attracted over 100,000 spectators.",
        "context": ["The First AFL-NFL World Championship Game was played on January 15, 1967, at the Los Angeles Memorial Coliseum."]
    }
]

print("📊 Faithfulness Comparison: Different Scenarios")
print("=" * 70)

for example in faithfulness_examples:
    sample = SingleTurnSample(
        user_input="Tell me about the first Super Bowl",
        response=example["response"],
        retrieved_contexts=example['context']
    )
    score = run_async(faithfulness_metric.single_turn_ascore(sample))

    print(f"\n{example['name']}: ")
    print(f"   Response: {example['response'][:80] if len(example['response'])> 80 else example['response']}")
    print(f"Score: { score:.2f}")

📊 Faithfulness Comparison: Different Scenarios

Perfect Faithfulness (No hallucinations): 
   Response: The first Super Bowl was played on January 15, 1967 at the Los Angeles Memorial 
Score: 1.00

Partial Faithfulness (Some hallucinations): 
   Response: The first Super Bowl was on January 15, 1967. The Green Bay Packers won 35-10 wi
Score: 0.33

Zero Faithfulness (Complete hallucination): 
   Response: The first Super Bowl was held in Miami in 1970 and attracted over 100,000 specta
Score: 0.00


In [26]:
original_question = "When was the first Super Bowl?"
test_answer = "The first Super Bowl was held on January 15, 1967"

print("📝 Original Question:")
print(f"   '{original_question}'")
print("\n📝 Answer to Evaluate:")
print(f"   '{test_answer}'")

📝 Original Question:
   'When was the first Super Bowl?'

📝 Answer to Evaluate:
   'The first Super Bowl was held on January 15, 1967'


In [28]:
question_gen_prompt = ChatPromptTemplate.from_template("""
Given the following answer, generate exactly 3 different questions that this answer would be a good response to.
The questions should be varied but all answerable by this response.

Answer: {answer}

Generate 3 questions (one per line):
1.
2.
3.
""")

question_gen_chain = question_gen_prompt | llm | StrOutputParser()

generated_question = question_gen_chain.invoke({"answer":test_answer})

print("🔍 STEP 1: Generated Hypothetical Questions")
print("=" * 50)
print(generated_question)

🔍 STEP 1: Generated Hypothetical Questions
1. When was the first Super Bowl held?  
2. What significant sports event took place on January 15, 1967?  
3. Can you tell me the date of the inaugural Super Bowl?  


In [30]:
generated_questions = [
    "When was the first Super Bowl held?",
    "What date was the inaugural Super Bowl?",
    "On what day did the first Super Bowl take place?"
]

print("📋 Questions for embedding comparison:")
print("   Original: ",original_question)
print("   Generated:")
for i,q in enumerate(generated_questions,1):
    print(f"      {i}. {q}")

📋 Questions for embedding comparison:
   Original:  When was the first Super Bowl?
   Generated:
      1. When was the first Super Bowl held?
      2. What date was the inaugural Super Bowl?
      3. On what day did the first Super Bowl take place?


In [33]:
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

print("✅ Cosine similarity function ready")
print("\n📐 Formula: cos(θ) = (A · B) / (||A|| × ||B||)")

✅ Cosine similarity function ready

📐 Formula: cos(θ) = (A · B) / (||A|| × ||B||)


In [35]:
print("🔍 STEP : Computing Embeddings and Similarities")
print("=" * 60)

original_embedding = embeddings.embed_query(original_question)
print(f"\n✅ Original question embedded (dim={len(original_embedding)})")

similarities = []
for i, gen_q in enumerate(generated_questions,1):
    gen_embedding = embeddings.embed_query(gen_q)
    sim = cosine_similarity(original_embedding, gen_embedding)
    similarities.append(sim)

    print(f"\n   Question {i}. '{gen_q}'")
    print(f"     Similarity to original: {sim:.4f}")

🔍 STEP : Computing Embeddings and Similarities

✅ Original question embedded (dim=1536)

   Question 1. 'When was the first Super Bowl held?'
     Similarity to original: 0.9821

   Question 2. 'What date was the inaugural Super Bowl?'
     Similarity to original: 0.9415

   Question 3. 'On what day did the first Super Bowl take place?'
     Similarity to original: 0.9574


In [39]:
manual_relavancy = np.mean(similarities)
print("🔢 STEP 3: Calculate Answer Relevancy Score")
print("=" * 50)
print(f"   Similarities: {[f'{s:.4f}'for s in similarities]}")

print(f"   Formula: Average of similarities")
print(f"\n  ({'+'.join([f'{s:.4f}' for s in similarities])})/{len(similarities)}")
print(f"\n   📊 Manual Answer Relevancy: {manual_relavancy:.4f}")

🔢 STEP 3: Calculate Answer Relevancy Score
   Similarities: ['0.9821', '0.9415', '0.9574']
   Formula: Average of similarities

  (0.9821+0.9415+0.9574)/3

   📊 Manual Answer Relevancy: 0.9604


In [ ]:
relevancy_sample = SingleTurnSample(
    user_input=original_question,
    response=test_answer,
    retrieved_contexts=["The First AFL-NFL World Championship Game was played on January 15, 1967."]
)

relevancy_metric = ResponseRelevancy(llm=ragas_llm, embeddings=ragas_embeddings)
ragas_relevancy = run_async(relevancy_metric.single_turn_ascore(relevancy_sample))

print("🔬 RAGAS Answer Relevancy Result")
print("=" * 50)

print(f" \n    Manual calculation: {manual_relavancy:.4f}")
print(f"     Ragas metric score: {ragas_relevancy:.4f}")

🔬 RAGAS Answer Relevancy Result

   Manual calculation: 0.9604
     Ragas metric score: 0.9821


In [45]:
relevancy_examples = [
    {
        "name": "Highly Relevant (Directly answers WHEN)",
        "question": "When was the first Super Bowl?",
        "answer": "The first Super Bowl was held on January 15, 1967.",
    },
    {
        "name": "Partially Relevant (Answers but adds extra info)",
        "question": "When was the first Super Bowl?",
        "answer": "The Super Bowl is the annual championship game of the NFL, first held on January 15, 1967.",
    },
    {
        "name": "Low Relevancy (Doesn't answer WHEN)",
        "question": "When was the first Super Bowl?",
        "answer": "The Super Bowl is the annual championship game of the National Football League.",
    },
    {
        "name": "Off-topic (Completely irrelevant)",
        "question": "When was the first Super Bowl?",
        "answer": "Pizza is a popular Italian dish that spread worldwide in the 20th century.",
    }
]

print("Answer Relevancy Comparison: ")
print("="*70)

for example in relevancy_examples:
    sample = SingleTurnSample(
        user_input=example['question'],
        response=example['answer'],
        retrieved_contexts=["Context not relevant for this metric."]
    )
    score = run_async(relevancy_metric.single_turn_ascore(sample))

    print(f"\n   {example['name']}")
    print(f"   Q: '{example['question']}'")
    print(f"   A: '{example['answer'][:60]}...'"  if len(example['answer'])>60 else f"   A: '{example['answer']}'")

    print(f"   Score: {score:.4f}")

Answer Relevancy Comparison: 

   Highly Relevant (Directly answers WHEN)
   Q: 'When was the first Super Bowl?'
   A: 'The first Super Bowl was held on January 15, 1967.'
   Score: 0.9821

   Partially Relevant (Answers but adds extra info)
   Q: 'When was the first Super Bowl?'
   A: 'The Super Bowl is the annual championship game of the NFL, f...'
   Score: 0.9455

   Low Relevancy (Doesn't answer WHEN)
   Q: 'When was the first Super Bowl?'
   A: 'The Super Bowl is the annual championship game of the Nation...'
   Score: 0.8970

   Off-topic (Completely irrelevant)
   Q: 'When was the first Super Bowl?'
   A: 'Pizza is a popular Italian dish that spread worldwide in the...'
   Score: 0.7797
